In [ ]:
import cv2
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
from scipy.ndimage import interpolation as inter

In [ ]:
pic=glob('images/*.jpg')

In [ ]:
img=cv2.imread(pic[0])


In [ ]:
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
def aspect_ratio_resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
        
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)


In [ ]:
new=aspect_ratio_resize(img,800)

In [ ]:
new.shape

In [ ]:
plt.imshow(new)

In [ ]:
def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1, dtype=float)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype=float)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
          borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated

In [ ]:
angle, rotated = correct_skew(new)

In [ ]:
plt.imshow(rotated)

In [ ]:
angle, rotated = correct_skew(img)
print(angle)
